In [1]:
import pandas as pd
import numpy as np
from surprise import SVDpp
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

from read_and_split_data import split_data, filter_animes_without_grade

In [2]:
def get_dataset():
    anime = pd.read_parquet("../datasets/anime.parquet")
    anime = anime[["anime_id", "type"]]
    users = pd.read_parquet("../datasets/users.parquet")
    base_df = users.merge(anime, on="anime_id", how="left")
    return base_df

base_df = get_dataset()
data = split_data(base_df)

In [3]:
kf = KFold(n_splits=5)

In [4]:
def predict_and_error(data, algo, path):
    fold = 0
    predictions_list = []
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        predictions_list.append(predictions)
        # Mean Squared Error
        accuracy.rmse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mae(predictions, verbose=True)
        # Fraction of Concordant Pairs
        accuracy.fcp(predictions, verbose=True)
        base_df = pd.DataFrame(columns=["user", "anime", "actual", "est", "details"], data=predictions)
        base_df.to_parquet(f"{path}_{fold}.parquet")
        fold += 1
    return predictions

In [5]:
algo_normal_predictor = SVDpp()

In [6]:
predictions = predict_and_error(data, algo_normal_predictor, path="../predictions/svdpp/anime_type_tv/no_filter/svdpp")

RMSE: 2.4634
MSE: 6.0682
MAE:  1.7884
FCP:  0.6425
RMSE: 2.5266
MSE: 6.3838
MAE:  1.8588
FCP:  0.6195
RMSE: 2.4953
MSE: 6.2267
MAE:  1.8247
FCP:  0.6283
RMSE: 2.5150
MSE: 6.3250
MAE:  1.8449
FCP:  0.6234
RMSE: 2.5130
MSE: 6.3152
MAE:  1.8437
FCP:  0.6242


In [7]:
base_df_without_negative = filter_animes_without_grade(base_df)

In [8]:
predictions_without_negative = predict_and_error(data, algo_normal_predictor, path="../predictions/svdpp/anime_type_tv/with_filter_remove_negative/svdpp")

RMSE: 2.4622
MSE: 6.0622
MAE:  1.7879
FCP:  0.6409
RMSE: 2.5423
MSE: 6.4632
MAE:  1.8765
FCP:  0.6090
RMSE: 2.5555
MSE: 6.5303
MAE:  1.8868
FCP:  0.6050
RMSE: 2.5349
MSE: 6.4257
MAE:  1.8647
FCP:  0.6155
RMSE: 2.5570
MSE: 6.5381
MAE:  1.8892
FCP:  0.6025
